In [2]:
#import data and libraries (do this by using the 1001 button above to the right)

from io import StringIO
import requests
import json
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Insert your data as a Pandas DataFrame

* Click on the cell below to highlight it.
* In the upper right of this notebook, click on `01/00` to open the Data tab, where you should see the `df_focusedvalues.csv` that you loaded earlier.
* Click on `Insert to code` -> `Insert Pandas DataFrame`
* Make sure that the data is called `df_data_1` and change it if it is not.

In [3]:
# @hidden_cell
# This function accesses a file in your Object Sto
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e4d75609fc03412bb5010f4da8f7e2a0 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Mb8cJUSMRm__NjkdK_kOSotCjkeresVWUtz9_WadyWha',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_e4d75609fc03412bb5010f4da8f7e2a0.get_object(Bucket='foodsecurity-donotdelete-pr-ujuci7icx7w4qw',Key='df_focusedvalues.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,State,County,PCT_REDUCED_LUNCH10,PCT_DIABETES_ADULTS10,PCT_OBESE_ADULTS10,FOODINSEC_10_12,PCT_OBESE_CHILD11,PCT_LACCESS_POP10,PCT_LACCESS_CHILD10,PCT_LACCESS_SENIORS10,...,WICS12,PCT_NHWHITE10,PCT_NHBLACK10,PCT_HISP10,PCT_NHASIAN10,PCT_65OLDER10,PCT_18YOUNGER10,POVRATE10,CHILDPOVRATE10,SNAP_PART_RATE10
0,AL,Autauga,6.886107,11.8,30.5,17.9,13.5,33.769657,8.837112,4.376378,...,5.0,77.246156,17.582599,2.400542,0.855766,11.995382,26.777959,11.9,17.5,79.0
1,AL,Baldwin,5.542340,11.8,26.6,17.9,13.3,19.318473,4.343199,3.513570,...,27.0,83.504787,9.308425,4.384824,0.735193,16.771185,22.987408,13.3,20.2,79.0
2,AL,Barbour,4.582140,14.2,37.3,17.9,18.6,20.840972,3.425062,2.805166,...,7.0,46.753105,46.691190,5.051535,0.389700,14.236807,21.906982,25.3,36.2,79.0
3,AL,Bibb,9.253986,11.1,34.3,17.9,13.7,4.559753,1.087518,0.657008,...,5.0,75.020729,21.924504,1.771765,0.096007,12.681650,22.696923,20.9,28.6,79.0
4,AL,Blount,8.097947,14.0,30.4,17.9,18.5,2.700840,0.671490,0.340269,...,6.0,88.887338,1.263040,8.070200,0.200621,14.722096,24.608353,16.5,24.1,79.0


In [4]:
df_data_1.shape

(3262, 32)

In [5]:
df_data_1.columns

Index(['State', 'County', 'PCT_REDUCED_LUNCH10', 'PCT_DIABETES_ADULTS10',
       'PCT_OBESE_ADULTS10', 'FOODINSEC_10_12', 'PCT_OBESE_CHILD11',
       'PCT_LACCESS_POP10', 'PCT_LACCESS_CHILD10', 'PCT_LACCESS_SENIORS10',
       'PCT_LOCLFARM07', 'FMRKT13', 'PCT_FMRKT_SNAP13', 'PCT_FMRKT_WIC13',
       'FMRKT_FRVEG13', 'PCT_FRMKT_FRVEG13', 'PCT_FRMKT_ANMLPROD13',
       'FOODHUB12', 'FARM_TO_SCHOOL', 'SODATAX_STORES11', 'GROC12', 'SNAPS12',
       'WICS12', 'PCT_NHWHITE10', 'PCT_NHBLACK10', 'PCT_HISP10',
       'PCT_NHASIAN10', 'PCT_65OLDER10', 'PCT_18YOUNGER10', 'POVRATE10',
       'CHILDPOVRATE10', 'SNAP_PART_RATE10'],
      dtype='object')

In [6]:
#remove NaNs
df_data_1 = df_data_1.dropna(how='any')

In [7]:
df_data_1.shape

(1937, 32)

Dividing the predictors and dependent attributes of the data set

In [8]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

X = df_data_1.iloc[:,:-1].values
y = df_data_1.iloc[:,-1].values

# Handle categorical variable - State and County columns
labelencoder_X = LabelEncoder()
X[:,1] = labelencoder_X.fit_transform(X[:,1])
X[:,2] = labelencoder_X.fit_transform(X[:,2])

hotonencoder_X = OneHotEncoder(categories = 'auto')
X = hotonencoder_X.fit_transform(X).toarray()

# Avoiding the dummy trap
X = X[:, 1:]

#X = pd.get_dummies(X)


<h3> Random Forest Regressor

In [9]:
# We are going to keep 30% of the dataset in test dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle = True)

In [10]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 25 decision trees
rf = RandomForestRegressor(n_estimators = 25, random_state = 42)

rf.fit(X_train, y_train)

# y_pred contains all the values predicted by trained model
y_pred = rf.predict(X_test)

In [11]:
#import metrics library
from sklearn import metrics 

#print result of MAE
print(metrics.mean_absolute_error(y_test, y_pred))

#print result of MSE
print(metrics.mean_squared_error(y_test, y_pred))

#print result of RMSE
print(np.sqrt(metrics.mean_absolute_error(y_test, y_pred)))


0.004261168384879757
0.00654295532646049
0.0652776254537476


In [12]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

2020-07-15 09:26:36,181 - watson_machine_learning_client.metanames - WARNING - 'AUTHOR_EMAIL' meta prop is deprecated. It will be ignored.


In [13]:
wml_credentials = {
  "apikey": "wHxKxpGNy7hnbVWus8MXrhbGRlWsOBzeeuI4lXBoXrKP",
  "iam_apikey_description": "Auto-generated for key a0327825-1ffb-4072-8a38-1c5dc7fbe212",
  "iam_apikey_name": "Service credentials-1",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/87454c2ca89e404aa44380d73fdc7fcc::serviceid:ServiceId-fe0bab3e-2fa0-4695-ac84-d89bb6293935",
  "instance_id": "a2cafc2c-53b0-4f98-b4f2-019a69c4d6ed",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}

In [14]:
client = WatsonMachineLearningAPIClient(wml_credentials)

In [15]:
model_props = {
    client.repository.ModelMetaNames.AUTHOR_NAME: "Nishita",
    client.repository.ModelMetaNames.AUTHOR_EMAIL: "nishita040mca18@igdtuw.ac.in",
    client.repository.ModelMetaNames.NAME: "Food Insecurity"
}

In [16]:
model_artifact = client.repository.store_model(rf, meta_props = model_props)

In [17]:
model_artifact

{'metadata': {'guid': 'aa18d527-10ee-431b-8f1d-b89542b43c0b',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a2cafc2c-53b0-4f98-b4f2-019a69c4d6ed/published_models/aa18d527-10ee-431b-8f1d-b89542b43c0b',
  'created_at': '2020-07-15T09:26:37.982Z',
  'modified_at': '2020-07-15T09:26:38.031Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'learning_configuration_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a2cafc2c-53b0-4f98-b4f2-019a69c4d6ed/published_models/aa18d527-10ee-431b-8f1d-b89542b43c0b/learning_configuration',
  'author': {'name': 'Nishita'},
  'name': 'Food Insecurity',
  'learning_iterations_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a2cafc2c-53b0-4f98-b4f2-019a69c4d6ed/published_models/aa18d527-10ee-431b-8f1d-b89542b43c0b/learning_iterations',
  'feedback_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a2cafc2c-53b0-4f98-b4f2-019a69c4d6ed/published_models/aa18d527-10ee-431b-8f1d-b89542b43c0b/feedback',
  'latest_version': {'url': '

In [18]:
guid = client.repository.get_model_uid(model_artifact)

In [19]:
guid

'aa18d527-10ee-431b-8f1d-b89542b43c0b'

In [20]:
deploy = client.deployments.create(guid, name = "Snap Prediction")



#######################################################################################

Synchronous deployment creation for uid: 'aa18d527-10ee-431b-8f1d-b89542b43c0b' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='8c7cb8a3-7d95-4d24-b529-fc7bfa1e8d33'
------------------------------------------------------------------------------------------------




In [21]:
deploy

{'metadata': {'guid': '8c7cb8a3-7d95-4d24-b529-fc7bfa1e8d33',
  'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a2cafc2c-53b0-4f98-b4f2-019a69c4d6ed/deployments/8c7cb8a3-7d95-4d24-b529-fc7bfa1e8d33',
  'created_at': '2020-07-15T09:32:02.284Z',
  'modified_at': '2020-07-15T09:32:02.616Z'},
 'entity': {'runtime_environment': 'python-3.6',
  'name': 'Snap Prediction',
  'scoring_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a2cafc2c-53b0-4f98-b4f2-019a69c4d6ed/deployments/8c7cb8a3-7d95-4d24-b529-fc7bfa1e8d33/online',
  'deployable_asset': {'name': 'Food Insecurity',
   'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a2cafc2c-53b0-4f98-b4f2-019a69c4d6ed/published_models/aa18d527-10ee-431b-8f1d-b89542b43c0b',
   'guid': 'aa18d527-10ee-431b-8f1d-b89542b43c0b',
   'created_at': '2020-07-15T09:32:02.252Z',
   'type': 'model'},
  'description': 'Description of deployment',
  'status_details': {'status': 'DEPLOY_SUCCESS'},
  'model_type': 'scikit-learn-0.20',
  'status':

In [22]:
scoring_url = client.deployments.get_scoring_url(deploy)

In [23]:
scoring_url

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/a2cafc2c-53b0-4f98-b4f2-019a69c4d6ed/deployments/8c7cb8a3-7d95-4d24-b529-fc7bfa1e8d33/online'